# Hippocampus segmentation

*This notebook should be run into a kaggle environment with GPU T4 x 2. If you want to use FastSurfer on windows see: https://deep-mi.org/FastSurfer/dev/overview/INSTALL.html*

- Download the fastsurfer package

In [ ]:
# Here we first setup the environment by downloading the open source deep-mi/fastsurfer project and the required packages
import os
import sys
from os.path import exists, join, basename, splitext

print("Starting setup. This could take a few minutes")
print("----------------------------------------------")

is_google_colab = "colab.research.google.com" in str(os.environ)
if is_google_colab:
    # this is for a Google Colab Notebook
    SETUP_DIR = "/content/"
else:
    # This is for Kaggle Notebook or local development
    SETUP_DIR = os.environ["HOME"] + "/fastsurfer_tutorial/"

# Go to the FastSurfer directory
!mkdir -p "{SETUP_DIR}"
%cd "{SETUP_DIR}"

print(f"Using {SETUP_DIR} to store files.")

print("Downloading FastSurfer")
print("----------------------------------------------")


git_repo_url = 'https://github.com/deep-mi/fastsurfer.git'
project_name = splitext(basename(git_repo_url))[0]
FASTSURFER_HOME = SETUP_DIR + project_name + "/"
if not exists(project_name):
  # clone and install dependencies
  ! git clone -q --branch stable $git_repo_url
  ! pip install -r $FASTSURFER_HOME/requirements.txt
sys.path.append(FASTSURFER_HOME)

# Update dependencies
print("Installing required packages")
print("----------------------------------------------")

! pip install torchio==0.18.83
! pip install yacs==0.1.8
! pip install plotly==5.9.0
! pip install monai
! pip install nilearn
! pip install nibabel

print("Finished setup")
print("----------------------------------------------")

Starting setup. This could take a few minutes
----------------------------------------------
/root/fastsurfer_tutorial
Using /root/fastsurfer_tutorial/ to store files.
----------------------------------------------
Installing required packages
----------------------------------------------
Finished setup
----------------------------------------------


In [ ]:
import nibabel as nib
import numpy as np
import shutil

from nilearn import plotting
from monai.transforms import (
	CropForeground,
    Resize, 
    SpatialPad,
    Compose,
    EnsureChannelFirst,
    Spacing,
    LoadImage,
)

2025-05-23 12:35:48.436176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748003748.459120    8114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748003748.465981    8114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


- Read a pickle list with the id of the subjects to download

In [ ]:
import pickle

with open('path_to_data', 'rb') as f:
    subject_names = pickle.load(f)

- Get the path of the image data (T1w) scans of the subjects inside `subject_names`

In [ ]:
import os

paths = []

for dirs in sorted(os.listdir('/kaggle/input/image_dataset/T1w')):
    if dirs in subject_names:
        walk = list(os.walk(os.path.join('/kaggle/input/image_dataset/T1w', dirs)))

        if len(walk) == 1:
            paths.append(walk[0][0] + '/' + walk[0][2][0])
        else:
            paths.append(walk[1][0] + '/' + walk[1][2][0])

- Plot an example

In [ ]:
plotting.plot_anat(nib.load(paths[0]))

- Define the transformation to perform after the segmentation

In [10]:
size = 40

transform = Compose([
    LoadImage(reader='NibabelReader'),
    EnsureChannelFirst(),
    Spacing(
        pixdim=(1.0, 1.0, 1.0),
        mode='bilinear',
        align_corners=True,
        scale_extent=True
    ),
    CropForeground(select_fn=(lambda x: x > 0), allow_smaller=True),
    SpatialPad(spatial_size=(size, size, size), mode='minimum'),
    Resize(
            spatial_size=(size, size, size),
            size_mode='all',
            mode='bilinear',
            align_corners=True,
            anti_aliasing=True
    ),
])

- Fastsurfer labels for hippocampus left and right

In [ ]:
hippo_left_id = 17
hippo_right_id = 53
biomarkers = [hippo_left_id, hippo_right_id]

In [ ]:
img_lst = paths
subject_names = subject_names

- Define the subprocess to run FastSurfer

In [ ]:
import subprocess

errors = []

for i, zipped in enumerate(zip(img_lst, subject_names)):
    img, name = zipped
    print(f"\n\n---Processing {name}")
    print(f'---Path {img}')
    print(f'---This is element {i + 1} of the batch\n\n')
    
    command = [
        os.path.join('./fastsurfer', "run_fastsurfer.sh"),
        "--t1", img,
        "--sd", f"{SETUP_DIR}fastsurfer_seg/",
        "--sid", f"{name}",
        "--seg_only",
        "--py", "python3",
        "--allow_root"
    ]

    flag = True

    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError:
            errors.append((img, name))
            flag = False
            print(f'---Error processing {name}\n\n')

    if flag:
        pred_data = nib.load(f"fastsurfer_seg/{name}/mri/aparc.DKTatlas+aseg.deep.mgz").get_fdata()
        # use orig_nu to remove bias field
        orig = nib.load(f'fastsurfer_seg/{name}/mri/orig.mgz').get_fdata()
        
        os.makedirs(f'segmented/{name}/mri')
    
        for label, direction in zip(biomarkers, ['left', 'right']):
            test_cond = np.in1d(pred_data, label).reshape(pred_data.shape)
            roi = np.where(test_cond, 1, 0)
            seg = orig * roi
            print(f"\n\n---Getting roi for hippocampus {direction}")
    
            save_path_segmentation = f'fastsurfer_seg/{name}/mri/seg.nii.gz'
            save_path_transformation = f'segmented/{name}/mri/hippocampus_{direction}.nii.gz'
            
            nib.save(nib.Nifti1Image(seg, affine=np.eye(4)), save_path_segmentation)
        
            hippo = transform(save_path_segmentation)
            
            nib.save(nib.Nifti1Image(hippo.numpy(), affine=np.eye(4)), save_path_transformation)
            print(f"\n\n---Saving transformations for hippocampus {direction}")

    # Save a subset of segmentations on zip file
    if i % 10 == 0:
        zip_dir_name = f'segmented_backup_bacth4_{i}'
        shutil.make_archive(zip_dir_name, 'zip', 'segmented')
        shutil.move(f'{zip_dir_name}.zip', '/kaggle/working')
        print(f'\n\n---Backup saved on kaggle/working for iteration {i + 1}')
     
    shutil.rmtree('./fastsurfer_seg')

- The segementations are zipped

In [ ]:
shutil.make_archive('segmented_final', 'zip', 'segmented')
shutil.move('segmented_final', '/kaggle/working')

'/kaggle/working/segmented_final_batch2.zip'